# Notes on how conda works

by Giacomo Vianello (giacomov@stanford.edu)

This is just a collection of notes for building a C++ package and releasing with Conda. If you are releasing a python-only package, things can be a lot easier. We are going to use the CERN ROOT package because it's big and complex, and allows to go into details.

# What is conda

Conda is a package manager (like yum, apt ...). Good thing is, contrary to these other pacakge managers, you can use it on Linux, Mac and Windows.

It can distribute python packages (which is what it was born for), but also compiled packages written in any other language (C/C++, Fortran...).

# How it works

Packages in conda are contained in channels

## Channels

Channels are indepenent software repositories that contain packages. The default channels contain things like numpy, scipy, sklearn... Channels can be open for free on [Anaconda Cloud](https://anaconda.org/).

## On the user side

The user needs to install either Anaconda (a large collection of packages containing also the Conda package manager), or Miniconda (containing only Conda and a small subset of necessary packages, like python). 

For Miniconda, the procedure is (for a Linux 64 bit system):

```bash
> wget https://repo.continuum.io/miniconda/Miniconda2-latest-Linux-x86_64.sh
> bash source ./Miniconda2-latest-Linux-x86_64.sh
```

or for Mac:

```bash
> wget https://repo.continuum.io/miniconda/Miniconda2-latest-MacOSX-x86_64.sh
> bash source ./Miniconda2-latest-Linux-x86_64.sh
```

Other installers can be downloaded from [here](https://conda.io/miniconda.html).

Then, a package can typically be installed as:

```bash
> conda install [-c channel] [package name]
```

Note that there is no need to set any environment variables (actually, things like `LD_LIBRARY_PATH` or `DYLD_LIBRARY_PATH` or `PYTHONPATH` should be unset, because they can interfere with Conda)

## On the provider side

Packages must be built with the `conda build` system and uploaded to a channel on Anaconda cloud.

### Conda recipes

Before you can build a package and distribute with conda you need to provide a conda recipe, consisting of two files:

1. `meta.yaml`: this file contains all information on the package needed to build and distribute the package, except for the build commands (unless they are very simple). In particular, it contains the name of the package, version, where to get the source code from, dependencies for the building and for the run environment, ecc. The full documentation can be found [here](https://conda.io/docs/user-guide/tasks/build-packages/define-metadata.html). This is an example of a `meta.yaml` file for the ROOT package:

```
package:
   name: root
   version: "5.34.36"

source:
  url: https://root.cern.ch/download/root_v5.34.36.source.tar.gz
  
build:
  number: 0

requirements:
 
 build:
     - python
     - gsl
     - fftw
     - cmake
     - cfitsio
     
 run:
     - python
     - gsl
     - fftw
     - cfitsio
 
 test:
     imports:
         - ROOT
 
```
The `package` section specifies name and version. The `source` section specifies where to get the source from. In this case we get a `.tar.gz` file from the ROOT website, but this can be also a git/svn path or even a local directory (see [here](https://conda.io/docs/user-guide/tasks/build-packages/define-metadata.html#source-section)). The `build` section allows to specify a build number. The `requirements` section specifies names of packages that must be present in the `build` or in the `run` environment. NOTE: it is usually a bad idea to include here a `gcc` or `libgcc` package (or `gfortran`/`libgfortran`). Indeed, conda should use instead the equivalent installed on the system used to build (most likely, a container used for universal binaries). Otherwise, you will need to include those also in the `run` environment potentially causing conflicts with other packages.

2. `build.sh`: this script contains the commands to actually build the package. This can be arbitrarily complex. This is an example for ROOT:


        #!/bin/bash

        cd build

        if [ "$(uname)" == "Darwin" ]; then

            cmake -Dall=ON -Dkrb5=OFF -Dcocoa=ON -Dgnuinstall=ON -Drpath=ON -Dsoversion=ON -DCMAKE_INSTALL_PREFIX=${PREFIX} -DCMAKE_INSTALL_SYSCONFDIR=${PREFIX}/etc/root ..

        else

            cmake -Dall=ON -Dkrb5=OFF -Dgnuinstall=ON -Drpath=ON -Dsoversion=ON -DCMAKE_INSTALL_PREFIX=${PREFIX} -DCMAKE_INSTALL_SYSCONFDIR=${PREFIX}/etc/root ..

        fi

        cmake --build . --target install -- -j 8

        # Install pyROOT in the site-packages so there is no need for
        # setting PYTHONPATH
        # Get path of site-packages for this python
        site_package=$(python -c "from distutils.sysconfig import get_python_lib; print(get_python_lib())")
    cp ${PREFIX}/lib/root/libPyROOT.* ${site_package}
    cp ${PREFIX}/lib/root/ROOT.py ${site_package}

In order for the build to work in the Conda environment of the user, you **have to build using [RPATH](https://en.wikipedia.org/wiki/Rpath)** and link explicitly all libraries to all their dependencies. Also, you need to install the  produced libraries into `$PREFIX/lib` (or subdirectories within that), and binaries into `$PREFIX/bin`. Python packages need to be under the appropriate `site-packages` directory for the installed python. In the recipe above you can find a trick to find out what that is for any python, so that it will work for python2/3. Moving your packages there allows python to find them without the need of messing up with `$PYTHONPATH`, which simplifies a lot things for the user. If your package has data, they can be installed into any subdirectory of $PREFIX and then can be found at runtime using `$CONDA_PREFIX` (which points to the root of the conda installation at runtime). So for example, the ROOT package installs data files into /etc by default. We provided a flag to cmake so that those files are instead installed under `$PREFIX/etc/root` . ROOT uses an internal mechanism to store that, but they could also be found at runtime under `$CONDA_PREFIX/etc/root` .

### Build process

The build process consists in compiling your package so that it is compatible with the Conda system and works when the user installs it. In order for the binaries to be compatible with most Linux distribution, we will build inside the docker container which is used to build python wheels, which can be found [here](https://github.com/pypa/manylinux). We still need to find an equivalent procedure for Mac.

